In [39]:
import pandas as pd

# Converts CEN scores to distance scores.

In [2]:
df = pd.read_csv('../data/roberta/Astro1_decomposed_clean_pairs_roberta.csv')
pairs_df = pd.read_csv('../data/Astro1_decomposed_clean_pairs.csv')
display(pairs_df.head(), df.head())

,Unnamed: 0,first,second
0,0,You never give up.,You never find it difficult to change your min...
1,1,You never give up.,"Whatever you have set your sights on, you refu..."
2,2,You never give up.,You are patient unless someone takes you too far.
3,3,You never give up.,You are usually slow to anger unless someone t...
4,4,You never give up.,You are reliable.


,Unnamed: 0,predicted_class,C,N,E
0,0,neutral,0.022798,0.880442,0.096760
1,1,entailment,0.001746,0.039801,0.958454
2,2,neutral,0.004882,0.990273,0.004845
3,3,neutral,0.052050,0.944104,0.003846
4,4,entailment,0.001493,0.161846,0.836661


In [3]:
df = df.drop(columns=['Unnamed: 0'])
pairs_df = pairs_df.drop(columns=['Unnamed: 0'])
display(df, pairs_df)

,predicted_class,C,N,E
0,neutral,0.022798,0.880442,0.096760
1,entailment,0.001746,0.039801,0.958454
2,neutral,0.004882,0.990273,0.004845
3,neutral,0.052050,0.944104,0.003846
4,entailment,0.001493,0.161846,0.836661
...,...,...,...,...
1076,entailment,0.042239,0.253214,0.704547
1077,contradiction,0.655430,0.302836,0.041734
1078,neutral,0.102161,0.781847,0.115993
1079,entailment,0.000774,0.081088,0.918138


,first,second
0,You never give up.,You never find it difficult to change your min...
1,You never give up.,"Whatever you have set your sights on, you refu..."
2,You never give up.,You are patient unless someone takes you too far.
3,You never give up.,You are usually slow to anger unless someone t...
4,You never give up.,You are reliable.
...,...,...
1076,You are sensitive.,You are gentle.
1077,You are sensitive.,You are romantic.
1078,You are sentimental.,You are gentle.
1079,You are sentimental.,You are romantic.


In [5]:
def gen_dist(row):
    '''
    Generate distance score based on Contradiction, Entailment and Neutrality probabilities.
    Assumes that C+E+N=1.
    '''
    C = row['C']
    E = row['E']
    N = row['N']
    
    if C > 0.5:
        return C
    elif C == E:
        return 0.5
    else:
        return N/(N+E)*0.5

In [6]:
ser_res = df.apply(lambda row: gen_dist(row), axis=1)
ser_res.name = 'dist'
df = df.join(ser_res)
df.head()

,predicted_class,C,N,E,dist
0,neutral,0.022798,0.880442,0.096760,0.450491
1,entailment,0.001746,0.039801,0.958454,0.019935
2,neutral,0.004882,0.990273,0.004845,0.497565
3,neutral,0.052050,0.944104,0.003846,0.497971
4,entailment,0.001493,0.161846,0.836661,0.081044


In [7]:
save_path = input("enter path to save csv file of pairwise distance scores:\n")
df.to_csv(save_path)

enter path to save csv file of pairwise distance scores:
 ../data/roberta/Astro1_decomposed_clean_pairs_roberta_distances.csv


### generate dist score from a pair of CEN scores, and generate a pair of indices for each sentence pair.

In [94]:
def gen_dist2(row):
    '''
    Generate distance score based on Contradiction, Entailment and Neutrality probabilities for both directions.
    Assumes that C+E+N=1.
    '''    
    max1 = max([row['C1'], row['E1'], row['N1']])
    max2 = max([row['C2'], row['E2'], row['N2']])
    if max1 > max2:
        C, E, N = row['C1'], row['E1'], row['N1']
    else:
        C, E, N = row['C2'], row['E2'], row['N2']
    
    if C > 0.5:
        return C
    elif C == E:
        return 0.5
    else:
        return N/(N+E)*0.5

In [95]:
df = pd.read_csv('../data/astro0_decomposed_clean_roberta.csv')
df.head()

,Unnamed: 0,C1,N1,E1,C2,N2,E2
0,0,0.014990,0.901193,0.083817,0.168603,0.702198,0.129199
1,1,0.021296,0.439075,0.539628,0.890627,0.093778,0.015595
2,2,0.002648,0.244821,0.752531,0.061536,0.715761,0.222702
3,3,0.001137,0.633402,0.365460,0.029134,0.667270,0.303596
4,4,0.001929,0.536462,0.461609,0.041221,0.582456,0.376323


In [96]:
ser_res = df.apply(lambda row: gen_dist2(row), axis=1)
ser_res.name = 'dist'
df = df.join(ser_res)
df.head()

,Unnamed: 0,C1,N1,E1,C2,N2,E2,dist
0,0,0.014990,0.901193,0.083817,0.168603,0.702198,0.129199,0.457454
1,1,0.021296,0.439075,0.539628,0.890627,0.093778,0.015595,0.890627
2,2,0.002648,0.244821,0.752531,0.061536,0.715761,0.222702,0.122736
3,3,0.001137,0.633402,0.365460,0.029134,0.667270,0.303596,0.343647
4,4,0.001929,0.536462,0.461609,0.041221,0.582456,0.376323,0.303749


In [97]:
pair_df = pd.read_csv("../data/roberta/astro0_decomposed_clean.csv")
pair_df.head()

,Unnamed: 0,first,second
0,0,"People with Virgo rising tend to be practical,...",You have an ingenious mind.
1,1,"People with Virgo rising tend to be practical,...",You have an active mind.
2,2,"People with Virgo rising tend to be practical,...",You have an alert mind.
3,3,"People with Virgo rising tend to be practical,...",Gaining knowledge and putting it to good use a...
4,4,"People with Virgo rising tend to be practical,...",You strive for perfection.


In [98]:
import numpy as np
import json

In [99]:
# first = np.unique(pair_df['first'].values)
# second = np.unique(pair_df['second'].values)
# combined_arr = np.concatenate((first, second))
# sentences = np.unique(combined_arr)
# print(len(sentences))
with open("../data/decomposer/lineariser_output_3_clean.json", encoding='utf-8') as f:
    # make sure to set encoding=utf-8 otherwise symbols like '-' get decoded wrongly. 
    sent_dict = json.load(f)
sent_dict

{'0': ['People with Virgo rising tend to be practical, analytical, discriminating, fastidious, careful, exacting, attentive to details, methodical, quiet, unassuming, shy, critical, thoughtful, and somewhat self-centered.'],
 '1': ['You have an ingenious mind.',
  'You have an active mind.',
  'You have an alert mind.'],
 '2': ['Gaining knowledge and putting it to good use are important to you.'],
 '3': ['You strive for perfection.',
  'You can be quite the person to live with or to be around because your standards for yourself and others are so high.'],
 '4': ['At times others can never be "good" enough to meet those high standards.'],
 '5': ["Finding fault with what's wrong with things is your forte."],
 '6': ['Sometimes, though, this can make relationships sour as you often turn your critical eye on the one you love and the things they do.'],
 '7': ['Pessimism and being too self-critical are two faults you should try to improve upon.'],
 '8': ['You may tend to worry too much, especi

In [100]:
sentences = []
for k, v in sent_dict.items():
    for sent in v:
        sentences.append(sent)
len(sentences)

88

In [101]:
sent_ser = pd.Series(data=sentences)
sent_ser.head()

0    People with Virgo rising tend to be practical,...
1                          You have an ingenious mind.
2                             You have an active mind.
3                              You have an alert mind.
4    Gaining knowledge and putting it to good use a...
dtype: object

In [102]:
sent_ser[sent_ser == "You love being involved in multiple things at once and this may be a problem – too may irons in the fire at the same time."].index[0]

53

In [103]:
print(sent_ser.iloc[53])

You love being involved in multiple things at once and this may be a problem – too may irons in the fire at the same time.


### Attach sentence index information to each sentence pair

In [104]:
index_pairs = list()
for i, row in pair_df.iterrows():
#     print(row['first'])
#     print(row['second'])
#     print()
    first_index = sent_ser[sent_ser == row['first']].index[0]
    second_index = sent_ser[sent_ser == row['second']].index[0]
    index_pairs.append([first_index, second_index])

In [105]:
len(index_pairs)

3828

In [106]:
zipped_index_pairs = list(zip(*index_pairs))
pair_df = pair_df.assign(first_index=zipped_index_pairs[0])
pair_df = pair_df.assign(second_index=zipped_index_pairs[1])
pair_df.head()

,Unnamed: 0,first,second,first_index,second_index
0,0,"People with Virgo rising tend to be practical,...",You have an ingenious mind.,0,1
1,1,"People with Virgo rising tend to be practical,...",You have an active mind.,0,2
2,2,"People with Virgo rising tend to be practical,...",You have an alert mind.,0,3
3,3,"People with Virgo rising tend to be practical,...",Gaining knowledge and putting it to good use a...,0,4
4,4,"People with Virgo rising tend to be practical,...",You strive for perfection.,0,5


In [ ]:
path = input('enter save path for csv of dataset of sentences:\n')
sent_ser.to_csv(path)

In [108]:
pair_df = pair_df.drop(['Unnamed: 0'], axis='columns')
pair_df.head()

,first,second,first_index,second_index
0,"People with Virgo rising tend to be practical,...",You have an ingenious mind.,0,1
1,"People with Virgo rising tend to be practical,...",You have an active mind.,0,2
2,"People with Virgo rising tend to be practical,...",You have an alert mind.,0,3
3,"People with Virgo rising tend to be practical,...",Gaining knowledge and putting it to good use a...,0,4
4,"People with Virgo rising tend to be practical,...",You strive for perfection.,0,5


In [109]:
final_df = pair_df.join(df)
final_df.head()

,first,second,first_index,second_index,Unnamed: 0,C1,N1,E1,C2,N2,E2,dist
0,"People with Virgo rising tend to be practical,...",You have an ingenious mind.,0,1,0,0.014990,0.901193,0.083817,0.168603,0.702198,0.129199,0.457454
1,"People with Virgo rising tend to be practical,...",You have an active mind.,0,2,1,0.021296,0.439075,0.539628,0.890627,0.093778,0.015595,0.890627
2,"People with Virgo rising tend to be practical,...",You have an alert mind.,0,3,2,0.002648,0.244821,0.752531,0.061536,0.715761,0.222702,0.122736
3,"People with Virgo rising tend to be practical,...",Gaining knowledge and putting it to good use a...,0,4,3,0.001137,0.633402,0.365460,0.029134,0.667270,0.303596,0.343647
4,"People with Virgo rising tend to be practical,...",You strive for perfection.,0,5,4,0.001929,0.536462,0.461609,0.041221,0.582456,0.376323,0.303749


In [110]:
final_df = final_df.drop(['Unnamed: 0'], axis='columns')
final_df.head()

,first,second,first_index,second_index,C1,N1,E1,C2,N2,E2,dist
0,"People with Virgo rising tend to be practical,...",You have an ingenious mind.,0,1,0.014990,0.901193,0.083817,0.168603,0.702198,0.129199,0.457454
1,"People with Virgo rising tend to be practical,...",You have an active mind.,0,2,0.021296,0.439075,0.539628,0.890627,0.093778,0.015595,0.890627
2,"People with Virgo rising tend to be practical,...",You have an alert mind.,0,3,0.002648,0.244821,0.752531,0.061536,0.715761,0.222702,0.122736
3,"People with Virgo rising tend to be practical,...",Gaining knowledge and putting it to good use a...,0,4,0.001137,0.633402,0.365460,0.029134,0.667270,0.303596,0.343647
4,"People with Virgo rising tend to be practical,...",You strive for perfection.,0,5,0.001929,0.536462,0.461609,0.041221,0.582456,0.376323,0.303749


In [112]:
path = input('enter path to save csv of processed dataset of sentence pairs:\n')
final_df.to_csv(path)

enter path to save csv of processed dataset of sentence pairs:
 ../data/astro0/astro0_decomposed_processed_pairs.csv
